In [1]:
'''a) Define the @log_transaction decorator.
b) Implement the BankAccount class with the deposit and withdraw methods decorated appropriately.
c) Write a short code snippet demonstrating the usage of the BankAccount class, including deposit and withdrawal operations, and show how the transactions are logged.'''
from functools import wraps
import datetime

def log_transaction(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        transaction_type = func.__name__  # Get the name of the method (deposit or withdraw)
        amount = args[0]  # Assuming the first argument is the amount
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        account_number = self.account_number  # Assuming each instance of BankAccount has an account number
        
        # Log the transaction details to a file
        with open("transaction.log", "a") as f:
            log_message = f"{timestamp} - {transaction_type.upper()} - Account: {account_number}, Amount: {amount}\n"
            f.write(log_message)
        
        return func(self, *args, **kwargs)
    return wrapper

class BankAccount:
    def __init__(self, account_number, initial_balance=0):
        self.account_number = account_number
        self.balance = initial_balance

    @log_transaction
    def deposit(self, amount):
        self.balance += amount
        print(f"Deposited {amount}. New balance: {self.balance}")

    @log_transaction
    def withdraw(self, amount):
        if self.balance >= amount:
            self.balance -= amount
            print(f"Withdrew {amount}. New balance: {self.balance}")
        else:
            print("Insufficient balance")

# Example usage
account1 = BankAccount("12345", 1000)
account1.deposit(500)
account1.withdraw(200)


Deposited 500. New balance: 1500
Withdrew 200. New balance: 1300


In [12]:
# 1. **Authorization Check:**
#     - Problem: You have a web application with various endpoints, and you want to restrict access to certain endpoints based on user roles or permissions.
#     - Solution: Use a decorator to check if the user is authorized to access the endpoint before executing the function associated with it.

    
def authorize(func):
    def wrapper(user_role):
        if user_role == "admin":
            return func()
        else:
            return "Unauthorized access"
    return wrapper

@authorize
def admin_panel():
    return "Admin Panel Access Granted"

print(admin_panel("admin"))  # Output: Admin Panel Access Granted
print(admin_panel("user"))   # Output: Unauthorized access



Admin Panel Access Granted
Unauthorized access


In [14]:
'''2. **Logging:**
    - Problem: You want to log function calls with their arguments and return values to track the application's behavior for debugging and auditing purposes.
    - Solution: Use a decorator to log information before and after executing the function.
'''
def log(func):
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with arguments: {args}, {kwargs}")
        result = func(*args, **kwargs)
        print(f"{func.__name__} returned: {result}")
        return result
    return wrapper

@log
def add(x, y):
    return x + y

print(add(3, 5))  # Output: 8

Calling add with arguments: (3, 5), {}
add returned: 8
8


In [20]:
'''3. **Rate Limiting:**
    - Problem: You want to restrict the number of times a function can be called within a specific time frame to prevent abuse or resource exhaustion.
    - Solution: Use a decorator to track the number of times the function has been called and enforce a limit.'''

  
import time

def rate_limit(limit_per_second):
    def decorator(func):
        last_called = 0
        def wrapper(*args, **kwargs):
            nonlocal last_called
            current_time = time.time()
            if current_time - last_called < 1 / limit_per_second:
                return "Rate limit exceeded"
            last_called = current_time
            return func(*args, **kwargs)
        return wrapper
    return decorator

@rate_limit(2)  # Allow 2 calls per second
def api_call():
    return "API call successful"

print(api_call())  # Output: API call successful
print(api_call())  # Output: API call successful
print(api_call())  # Output: Rate limit exceeded


API call successful
Rate limit exceeded
Rate limit exceeded


In [17]:
'''4. **Caching:**
    - Problem: You have a function that performs expensive computations, and you want to cache its results to improve performance by avoiding redundant calculations.
    - Solution: Use a decorator to store the results of function calls in a cache and return the cached result if the same inputs are provided again.
'''
def cache(func):
        cached_results = {}
        def wrapper(*args):
            if args in cached_results:
                return cached_results[args]
            result = func(*args)
            cached_results[args] = result
            return result
        return wrapper

@cache
def fibonacci(n):
        if n <= 1:
          return n
        return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(5))  # Output: 5
   

5


In [19]:
# 5. **Retry Mechanism:**
#     - Problem: You have a function that interacts with external services, and you want to retry the function call a few times if it fails due to transient errors like network issues.
#     - Solution: Use a decorator to retry the function call with exponential backoff until it succeeds or the maximum number of retries is reached.

    
    
    
    
import time

def retry(max_retries, delay=1, backoff=2):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for attempt in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {attempt + 1}: {e}")
                    time.sleep(delay * (backoff ** attempt))
            return "Max retries exceeded"
        return wrapper
    return decorator

@retry(max_retries=3)
def api_call():
    # Simulate a transient error
    raise ConnectionError("Failed to connect")

print(api_call())  # Output: Attempt 1: Failed to connect, Attempt 2: Failed to connect, Attempt 3: Failed to connect, Max retries exceeded



Attempt 1: Failed to connect
Attempt 2: Failed to connect
Attempt 3: Failed to connect
Max retries exceeded
